## # Goal: Clean the CBO Excel file, calculate percent change, and divide by number of employed people in each year

#### Last edited: 10/17/21

In [ ]:
import pandas as pd
import numpy as np
import janitor

%load_ext nb_black

from IPython.core.display import display, HTML

display(HTML("<style>.container {width:90% !important;}</style>"))

In [205]:
# import
df = pd.read_csv(
    "/Users/samsloate/Desktop/807 MP/data/cbo/short_term_economic_projections_raw.csv",
    encoding="latin-1",
    skiprows=6,
    sep=",",
)

<IPython.core.display.Javascript object>

In [206]:
# clean columns
df = df.dropna(how="all").clean_names()
df = df.rename(
    columns={
        "unnamed_0": "component",
        "unnamed_1": "subcomponent",
        "unnamed_2": "subcomponent_1",
    }
)

# forward fill units
cols = ["component", "subcomponent", "subcomponent_1"]
df.loc[:, cols] = df.loc[:, cols].ffill()
df = df[~df.units.isna()]

<IPython.core.display.Javascript object>

In [207]:
# create income dataset: just millions/billions as units, and also include LFPR
inc = df[
    ((df.component == "Income") & (df.units == "Billions of dollars"))
    | (
        (
            (
                df.subcomponent
                == "Employment, Civilian, 16 Years or Older (Household Survey)"
            )
            & (df.units == "Millions")
        )
    )
]
inc = inc.copy().reset_index(drop=True)

# fix forward fill error
inc.loc[
    inc.subcomponent == "Profits, Corporate, With IVA & CCAdj", "subcomponent_1"
] = np.nan

inc.loc[
    inc.subcomponent == "Profits, Corporate, Domestic, With IVA & CCAdj",
    "subcomponent_1",
] = np.nan

# clean income dataset
inc.drop(
    "component", axis=1, inplace=True
)  # drop redundant column (only says 'income')

inc = inc.rename(
    columns={"subcomponent": "comp", "subcomponent_1": "subcomp"}
)  # rename
inc = inc[
    inc.columns.drop(list(inc.filter(regex="unnamed")))
]  # remove "unnamed" columns at end


inc.comp.replace(
    "Employment, Civilian, 16 Years or Older (Household Survey)",
    "employment",
    inplace=True,
)  # rename "employment"

val_vars = [str(i) for i in range(2017, 2032)]

inc = pd.melt(
    inc,
    id_vars=["comp", "subcomp", "units"],
    value_vars=val_vars,
    var_name="year",
    value_name="dollars",
)  # make long, not wide

<IPython.core.display.Javascript object>

In [208]:
# create new employment column
inc["employment"] = np.nan
inc.loc[inc.comp == "employment", "employment"] = inc.dollars
inc.employment = inc.employment.ffill()

# employment in millions
inc.employment = inc.employment * 1000000

# drop employment rows
inc = inc[inc.comp != "employment"]

# dollars in billions
inc.dollars = inc.dollars * 1000000000

# delete units column
inc.drop("units", axis=1, inplace=True)

# fill in "total" for sub component
inc.loc[inc.subcomp.isna(), "subcomp"] = inc.comp

<IPython.core.display.Javascript object>

In [209]:
# calculations
inc["dollars_per_wkr"] = (
    inc.dollars / inc.employment
)  # dollars per worker, to account for pop. growth

<IPython.core.display.Javascript object>

In [210]:
inc.sort_values(["comp", "subcomp"], inplace=True, ascending=[True, False])
inc["pct_change"] = inc.dollars_per_wkr.pct_change()
inc

,comp,subcomp,year,dollars,employment,dollars_per_wkr,pct_change
2,"Compensation of Employees, Paid","Compensation of Employees, Paid",2017,1.042260e+13,153335000.0,67972.739427,NaN
14,"Compensation of Employees, Paid","Compensation of Employees, Paid",2018,1.095010e+13,155758000.0,70302.006960,0.034268
26,"Compensation of Employees, Paid","Compensation of Employees, Paid",2019,1.143250e+13,157536000.0,72570.713996,0.032271
38,"Compensation of Employees, Paid","Compensation of Employees, Paid",2020,1.144860e+13,147793000.0,77463.749975,0.067424
50,"Compensation of Employees, Paid","Compensation of Employees, Paid",2021,1.231150e+13,153034000.0,80449.442608,0.038543
...,...,...,...,...,...,...,...
123,Wages and Salaries,Wages and Salaries,2027,1.284320e+13,160924000.0,79809.102433,0.036048
135,Wages and Salaries,Wages and Salaries,2028,1.333420e+13,161329000.0,82652.219998,0.035624
147,Wages and Salaries,Wages and Salaries,2029,1.382350e+13,161612000.0,85535.108779,0.034880
159,Wages and Salaries,Wages and Salaries,2030,1.430890e+13,161873000.0,88395.841184,0.033445


<IPython.core.display.Javascript object>

In [211]:
# subset to just the data we need
inc.replace("Wages and Salaries", "wages and salaries", inplace=True)
inc.replace("Nonwage Income", "nonwage inc", inplace=True)
inc.replace("Income, Personal", "total personal inc", inplace=True)

# round numbers
inc["pct_change"] = inc["pct_change"].round(4)
inc["dollars_per_wkr"] = inc["dollars_per_wkr"].round(2)
inc["employment"] = inc["employment"].round(0)

inc_all = inc

inc_subset = inc[
    (inc.comp == "wages and salaries")
    | (inc.comp == "nonwage inc")
    | (inc.comp == "total personal inc")
]

<IPython.core.display.Javascript object>

In [213]:
# export

inc_all.to_csv(
    "/Users/samsloate/Desktop/807 MP/data/cbo/short_term_economic_projections_clean_all.csv"
)
inc_subset.to_csv(
    "/Users/samsloate/Desktop/807 MP/data/cbo/short_term_economic_projections_clean_subset.csv"
)

<IPython.core.display.Javascript object>